# Menu Extraction Data Explorer

This notebook helps you explore the menu upload pipeline data.

## Tables:
- `menu_upload` - Upload tracking records
- `menu_upload_stage` - Pipeline stage status
- `menu_upload_recipe` - Links uploads to created recipes
- `recipe` - Extracted meal/dish information
- `recipe_ingredient` - Ingredients for each recipe
- `restaurant` - Restaurant information
- `menu` - Menu information


In [ ]:
# Setup: Import libraries
import asyncio
import pandas as pd
from sqlalchemy import text
import sys
sys.path.append('.')

from app.database import AsyncSessionLocal

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 100)
pd.set_option('display.width', None)

print("✅ Setup complete!")


/var/folders/s_/nn0tqk3548d9xbrc07r8f4_r0000gn/T/ipykernel_62232/1735630722.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


ImportError: cannot import name 'get_async_session' from 'app.database' (/Users/alexanderkok/feeb/apps/api/app/database.py)

In [ ]:
# Helper function to query the database
async def query_db(sql: str):
    """Execute a SQL query and return results as a pandas DataFrame."""
    async with AsyncSessionLocal() as session:
        result = await session.execute(text(sql))
        rows = result.fetchall()
        columns = result.keys()
        return pd.DataFrame(rows, columns=columns)

print("✅ Helper functions loaded!")


## 1. Menu Uploads Overview


In [ ]:
# View all menu uploads
uploads = await query_db("SELECT * FROM menu_upload ORDER BY created_at DESC")
print(f"Total uploads: {len(uploads)}")
uploads.head(10)


In [ ]:
# Upload status breakdown
status_counts = await query_db("""
    SELECT status, COUNT(*) as count 
    FROM menu_upload 
    GROUP BY status
""")
status_counts


## 2. Pipeline Stages


In [ ]:
# View stage progress
stages = await query_db("""
    SELECT 
        mu.id as upload_id,
        mu.source_type,
        mus.stage,
        mus.status,
        mus.started_at,
        mus.completed_at,
        mus.error_message
    FROM menu_upload mu
    LEFT JOIN menu_upload_stage mus ON mu.id = mus.menu_upload_id
    ORDER BY mu.created_at DESC, mus.stage
""")
stages.head(20)


## 3. Extracted Recipes (Meals)


In [ ]:
# View all recipes
recipes = await query_db("""
    SELECT 
        r.*,
        rest.name as restaurant_name
    FROM recipe r
    LEFT JOIN restaurant rest ON r.restaurant_id = rest.id
    ORDER BY r.created_at DESC
""")
print(f"Total recipes: {len(recipes)}")
recipes.head(10)


## 4. Recipe Ingredients (Enriched Data)


In [ ]:
# View recipe ingredients with ingredient names
recipe_ingredients = await query_db("""
    SELECT 
        r.name as recipe_name,
        i.name as ingredient_name,
        ri.quantity,
        ri.unit,
        ri.notes,
        ri.allergens,
        ri.confirmed
    FROM recipe_ingredient ri
    JOIN recipe r ON ri.recipe_id = r.id
    JOIN ingredient i ON ri.ingredient_id = i.id
    ORDER BY r.name, i.name
""")
print(f"Total recipe-ingredient links: {len(recipe_ingredients)}")
recipe_ingredients.head(20)


In [ ]:
# Count ingredients per recipe
ingredients_per_recipe = await query_db("""
    SELECT 
        r.name as recipe_name,
        COUNT(ri.ingredient_id) as ingredient_count
    FROM recipe r
    LEFT JOIN recipe_ingredient ri ON r.id = ri.recipe_id
    GROUP BY r.id, r.name
    ORDER BY ingredient_count DESC
""")
ingredients_per_recipe.head(10)


## 5. Database Summary


In [ ]:
# Get counts of all menu-related tables
summary = await query_db("""
    SELECT 'menu_upload' as table_name, COUNT(*) as count FROM menu_upload
    UNION ALL
    SELECT 'menu_upload_stage', COUNT(*) FROM menu_upload_stage
    UNION ALL
    SELECT 'menu_upload_recipe', COUNT(*) FROM menu_upload_recipe
    UNION ALL
    SELECT 'recipe', COUNT(*) FROM recipe
    UNION ALL
    SELECT 'recipe_ingredient', COUNT(*) FROM recipe_ingredient
    UNION ALL
    SELECT 'restaurant', COUNT(*) FROM restaurant
    UNION ALL
    SELECT 'menu', COUNT(*) FROM menu
""")

print("\n📊 MENU EXTRACTION DATABASE SUMMARY")
print("="*50)
summary
